In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from geopy.distance import distance, lonlat
from tqdm.notebook import tqdm

In [ ]:
# Creating the DataFrame of the earthquakes' probabilities
df = pd.read_table('incremental_annual_rates/incremental_annual_rates_USA.txt', sep='\s+', header=None)

df.columns = ['Lat', 'Long'] + [str(np.around(x,1)) for x in np.arange(4.6, 8.7, 0.1)]
df.drop(df.columns[42:], axis=1, inplace=True)
df.fillna(value=0.0, inplace=True)
df = df[df['4.6'] != 0.0]
df.rename(columns={'Lat':'Long', 'Long':'Lat'}, inplace=True)

df.to_csv('incremental_annual_rates/incremental_annual_rates_italy_without_zeros.csv', index=False)

In [ ]:
df = pd.read_csv('earthquake_probabilities/italy.csv')
df.head(2)

In [ ]:
g = nx.read_gml('networks/italy.gml')

In [ ]:
def intensity_europe(M, R):
    h = 3.91
    D = np.sqrt(h**2 + R**2)
    return 1.621*M - 1.343 - 0.0086*(D-h) - 1.037*(np.log(D)-np.log(h))

In [ ]:
l = [tuple(d.values()) for d in g.edges[('Rome', 'Pescara', 0)]['points']['point']]
edge = LineString(l)
point = Point(10.8,48.3)
nearest_point = nearest_points(edge, point)[0]
nearest_point.coords[:]

In [ ]:
for e in g.edges:
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    edge = LineString(edge_points)
    epicenter = Point(10.5,48.2)
    nearest_point = nearest_points(edge, epicenter)[0].coords[0]
    print(nearest_point)
    print(epicenter.coords[0])
    dist = distance(lonlat(*epicenter.coords[0]), lonlat(*nearest_point)).km
    intensity = intensity_europe(8.6, dist)
    print("%.1f" % intensity)

In [ ]:
#df['Epicenter'] = list(zip(df.Lat, df.Long))
df.insert(loc=0, column='Epicenter', value=list(zip(df.Lat, df.Long)))
df.set_index('Epicenter', inplace=True)
df.drop(['Lat', 'Long'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
epicenters = df.index.to_numpy()
magnitudes = df.columns.to_numpy()

In [54]:
I = np.ones((g.number_of_edges(), len(epicenters[:2]), len(magnitudes)))
for i, e in tqdm(enumerate(g.edges), total=g.number_of_edges()):
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    l = LineString(edge_points)
    for j, epicenter in tqdm(enumerate(epicenters[:2]), total=len(epicenters)):
        p = Point(epicenter)
        nearest_point = nearest_points(l, p)[0].coords[0]
        R = distance(lonlat(epicenter), lonlat(*nearest_point)).km
        for k, M in enumerate(magnitudes):
            intensity = intensity_europe(np.float(M), R)
            if intensity > 1.0:
                I[i,j,k] = intensity

(13.016076136662997, 42.40828181839583)


TypeError: lonlat() missing 1 required positional argument: 'y'

In [ ]:
list(enumerate(magnitudes[:10]))

In [ ]:
I[I>4]